In [1]:
# -*- coding: utf-8 -*-
"""
Skin Disease Prediction Script

This script loads a trained EfficientNet-B0 model and predicts the class
of a single skin lesion image. It requires the saved model weights (.pth)
and the label encoder (.pkl) generated by the training script.
"""
import os
import cv2
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
import joblib
import warnings

warnings.filterwarnings('ignore')

In [2]:
# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
# --- Paths to the trained model and artifacts ---
OUTPUT_DIR = r"C:\Users\vaghe\Downloads\Models\Gemini"
MODEL_PATH = os.path.join(OUTPUT_DIR, "skin_disease_efficientnet.pth")
ENCODER_PATH = os.path.join(OUTPUT_DIR, "label_encoder.pkl")

In [16]:


# --- Path to the image you want to classify ---
# 👈 *** POINT THIS TO THE IMAGE YOU WANT TO PREDICT ***
IMAGE_TO_PREDICT = r"D:\DermaNet\test\Melanoma Skin Cancer Nevi and Moles\atypical-nevi-36.jpg"

# --- Model & Image Parameters (must match training script) ---
IMG_SIZE = 224
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cuda


In [17]:
# ==============================================================================
# 2. MODEL AND TRANSFORMS DEFINITION
# (Must be identical to the training script)
# ==============================================================================

class SkinClassifier(nn.Module):
    """EfficientNet-B0 model with a custom classifier head."""
    def __init__(self, num_classes, dropout_rate=0.4):
        super().__init__()
        self.backbone = efficientnet_b0(weights=None) # Weights are loaded later

        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(p=dropout_rate / 2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

def get_inference_transforms():
    """
    Defines the exact same transformations used for validation/testing
    during training. This consistency is crucial.
    """
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])


# ==============================================================================
# 3. PREDICTION FUNCTION
# ==============================================================================

def predict_image(image_path, model, label_encoder, transform, device):
    """
    Loads an image, preprocesses it, and returns the predicted class
    and confidence score.

    Args:
        image_path (str): Path to the input image.
        model (torch.nn.Module): The trained model.
        label_encoder (LabelEncoder): The fitted label encoder.
        transform (A.Compose): The Albumentations transform pipeline.
        device (torch.device): The device to run inference on (CPU or CUDA).

    Returns:
        tuple: A tuple containing the predicted label (str) and the
               confidence score (float).
    """
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found or could not be read at {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None

    # Apply transformations
    preprocessed_image = transform(image=image)['image']
    
    # Add batch dimension and send to device
    image_tensor = preprocessed_image.unsqueeze(0).to(device)

    # Perform prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        
        # Apply softmax to get probabilities
        probabilities = torch.softmax(outputs, dim=1)
        
        # Get the top prediction
        confidence, predicted_idx = torch.max(probabilities, 1)

    # Decode the prediction using the label encoder
    predicted_label = label_encoder.inverse_transform([predicted_idx.item()])[0]
    confidence_score = confidence.item()

    return predicted_label, confidence_score


# ==============================================================================
# 4. MAIN EXECUTION
# ==============================================================================

if __name__ == '__main__':
    print("🚀 Starting Skin Disease Prediction")
    print(f"Using device: {DEVICE}")

    # --- 1. Check if required files exist ---
    if not os.path.exists(MODEL_PATH) or not os.path.exists(ENCODER_PATH):
        print(f"Error: Could not find required files.")
        print(f"  - Model expected at: {MODEL_PATH}")
        print(f"  - Encoder expected at: {ENCODER_PATH}")
        print("Please ensure the training process was completed and these files are in the correct directory.")
    else:
        # --- 2. Load the label encoder ---
        print(f"Loading label encoder from {ENCODER_PATH}...")
        label_encoder = joblib.load(ENCODER_PATH)
        num_classes = len(label_encoder.classes_)
        print(f"Found {num_classes} classes: {', '.join(label_encoder.classes_)}")

        # --- 3. Initialize model and load trained weights ---
        print(f"Loading model weights from {MODEL_PATH}...")
        model = SkinClassifier(num_classes=num_classes)
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        model.to(DEVICE)
        
        # --- 4. Get the necessary transformations ---
        inference_transform = get_inference_transforms()
        
        # --- 5. Make the prediction ---
        print("\n" + "="*50)
        print(f"🔬 Analyzing image: {IMAGE_TO_PREDICT}")
        
        if not os.path.exists(IMAGE_TO_PREDICT):
            print(f"\n❌ ERROR: The image file was not found at '{IMAGE_TO_PREDICT}'")
            print("Please update the 'IMAGE_TO_PREDICT' variable in the script.")
        else:
            predicted_class, confidence = predict_image(
                IMAGE_TO_PREDICT, model, label_encoder, inference_transform, DEVICE
            )
            
            # --- 6. Display the result ---
            if predicted_class and confidence is not None:
                print("\n--- Prediction Result ---")
                print(f"🩺 Predicted Disease : {predicted_class}")
                print(f"✅ Confidence        : {confidence * 100:.2f}%")
                print("="*50)
                print("\nDisclaimer: This is an AI-based prediction and not a medical diagnosis. Please consult a qualified dermatologist for any health concerns.")

🚀 Starting Skin Disease Prediction
Using device: cuda
Loading label encoder from C:\Users\vaghe\Downloads\Models\Gemini\label_encoder.pkl...
Found 9 classes: 1. Eczema 1677, 10. Warts Molluscum and other Viral Infections - 2103, 2. Melanoma 15.75k, 4. Basal Cell Carcinoma (BCC) 3323, 5. Melanocytic Nevi (NV) - 7970, 6. Benign Keratosis-like Lesions (BKL) 2624, 7. Psoriasis pictures Lichen Planus and related diseases - 2k, 8. Seborrheic Keratoses and other Benign Tumors - 1.8k, 9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k
Loading model weights from C:\Users\vaghe\Downloads\Models\Gemini\skin_disease_efficientnet.pth...

🔬 Analyzing image: D:\DermaNet\test\Melanoma Skin Cancer Nevi and Moles\atypical-nevi-36.jpg

--- Prediction Result ---
🩺 Predicted Disease : 8. Seborrheic Keratoses and other Benign Tumors - 1.8k
✅ Confidence        : 99.50%

Disclaimer: This is an AI-based prediction and not a medical diagnosis. Please consult a qualified dermatologist for any healt

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import pickle

class EfficientNetModel(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetModel, self).__init__()
        self.base_model = torch.hub.load('pytorch/vision:v0.10.0', 'efficientnet_b0', pretrained=True)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

def predict_image(image_path, model_path, encoder_path):
    # Load the model
    num_classes = 7  # Update with the number of classes in your model
    model = EfficientNetModel(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()

    # Load the label encoder
    with open(encoder_path, 'rb') as f:
        label_encoder = pickle.load(f)

    # Image transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load and transform the image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)

    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted_class = torch.max(probabilities, 1)
        
    # Decode the prediction
    predicted_label = label_encoder.inverse_transform([predicted_class.item()])[0]
    
    return {
        'predicted_class': predicted_label,
        'confidence': confidence.item()
    }

if __name__ == '__main__':
    # Example usage:
    image_path = '../uploads/c81bddfd-1f4d-472a-82a7-a8dd6f056fc7_skin_image.jpg'  # Provide a path to a test image
    model_path = 'skin_disease_efficientnet_model1.pth'
    encoder_path = 'label_encoder_mode1.pkl'
    
    try:
        result = predict_image(image_path, model_path, encoder_path)
        print(f"Prediction: {result['predicted_class']}")
        print(f"Confidence: {result['confidence']:.4f}")
    except FileNotFoundError:
        print(f"Error: Make sure the paths to the image, model, and encoder are correct.")
    except Exception as e:
        print(f"An error occurred: {e}")


🔍 Checking model files...


🔍 Checking model files...


NameError: name 'MODEL_PATH' is not defined